a. Load the data

In [91]:
import pandas as pd
import numpy as np
from sklearn import datasets, model_selection, preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
df_1 = pd.read_csv("sample_diabetes_mellitus_data.csv")
df_1.shape

(10000, 53)

b.  Split the data between train and test. (you can use train_test_split from sklearn or any other way)

In [92]:
train_df, test_df = train_test_split(df_1, test_size=0.3, random_state=42)

c. Remove those rows that contain NaN values in the columns: age, gender, ethnicity

In [93]:
df_2 = df_1.dropna(subset=['age', 'gender', 'ethnicity'])
df_2.shape

(9368, 53)

d. Fill NaN with the mean value of the column in the columns: height, weight.

In [ ]:
df_2['height'].fillna(df_2['height'].mean(), inplace=True)
df_2['weight'].fillna(df_2['weight'].mean(), inplace=True)

e. Generate dummies for ethnicity column (One hot encoding).

In [100]:
from sklearn.preprocessing import OneHotEncoder
# Apply one-hot encoding
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df_2[['ethnicity']]).toarray()
# Convert the encoded data back to a dataframe
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['ethnicity']))
# Concatenate the encoded dataframe with the 'id' column
encoded_df = pd.concat([df_2['encounter_id'].reset_index(drop=True), encoded_df], axis=1)
encoded_df.shape

(9368, 7)

In [101]:
df_3 = pd.merge(df_2, encoded_df, on='encounter_id', how='left')
df_3.shape

(9368, 59)

f. Create a binary variable for gender M/F.

In [102]:
df_3['gender_binary'] = df_3['gender'].map({'M': 1, 'F': 0})
df_3.shape

(9368, 60)

g. Train a model (for instance LogisticRegression or RandomForestClassifier from sklearn) in the train data. Use as features the columns: ‘age’, ‘height’, ‘weight’, ‘aids’, ‘cirrhosis’, ‘hepatic_failure’, ‘immunosuppression’, ‘leukemia’, ‘lymphoma’, ‘solid_tumor_with_metastasis’. Use as target the column: ‘diabetes_mellitus’

In [107]:
features = ['age', 'height', 'weight', 'aids', 'cirrhosis', 'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']
X = df_3[features]
y = df_3['diabetes_mellitus']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

c:\Users\Soledad.Acuna\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


h.  Predict the targets for both the train and test sets and add the prediction as a new column (use predict_proba from the model to get the predicted probabilities) name the new column something like predictions

In [113]:
train_df, test_df = train_test_split(df_3, test_size=0.3, random_state=42)

train_prob = model.predict_proba(X_train)[:, 1]  
train_df['predictions'] = train_prob  

test_prob = model.predict_proba(X_test)[:, 1]  
test_df['predictions'] = test_prob  

i.  Compute the train and test roc_auc metric using roc_auc_score from sklearn

In [117]:
from sklearn.metrics import roc_auc_score

train_ra = roc_auc_score(y_train, train_prob)
test_ra = roc_auc_score(y_test, test_prob)

print(f"ROC AUC of train: {train_ra:.4f}")
print(f"ROC AUC of test: {test_ra:.4f}")

ROC AUC of train: 0.6712
ROC AUC of test: 0.6708
